In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50V2 as ResNet
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
batch_size = 64

precision = tf.keras.metrics.Precision(name='precision')
recall = tf.keras.metrics.Recall(name='recall')
auc_roc = tf.keras.metrics.AUC(num_thresholds=200, name='auc_roc')
train_dir = "train/"
test_dir = "test/"
num_classes = 7

In [3]:
def plot(history):
    # Plot loss
    plt.plot(history.history['loss'], label='train_loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    # Plot accuracy
    plt.plot(history.history['accuracy'], label='train_acc')
    plt.plot(history.history['val_accuracy'], label='val_acc')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

    # Plot other performance metrics, such as precision, recall, f1_score, and auc_roc
    plt.plot(history.history['precision'], label='precision')
    plt.plot(history.history['recall'], label='recall')
    plt.plot(history.history['auc_roc'], label='auc_roc')
    plt.title('Training and Validation Performance Metrics')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

In [4]:
datagen = ImageDataGenerator(rescale=1 / 255)


train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  
    batch_size=64,
    class_mode='categorical' 
)


test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [ ]:
resnet = ResNet(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

x = resnet.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)

predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

model = tf.keras.models.Model(inputs=resnet.input, outputs=predictions)
for layer in resnet.layers:
    layer.trainable = False
    
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy', precision, recall, auc_roc])

history = model.fit(train_generator,
          epochs=50,
          validation_data=test_generator,
          callbacks=[early_stopping])

Epoch 1/50
449/449 [==============================] - 5146s 11s/step - loss: 1.4985 - accuracy: 0.4362 - precision: 0.6396 - recall: 0.2059 - auc_roc: 0.8015 - val_loss: 1.3235 - val_accuracy: 0.4957 - val_precision: 0.7689 - val_recall: 0.2267 - val_auc_roc: 0.8498
Epoch 2/50
449/449 [==============================] - 4881s 11s/step - loss: 1.3417 - accuracy: 0.4925 - precision: 0.6993 - recall: 0.2665 - auc_roc: 0.8436 - val_loss: 1.2700 - val_accuracy: 0.5134 - val_precision: 0.7496 - val_recall: 0.2887 - val_auc_roc: 0.8614
Epoch 3/50
449/449 [==============================] - ETA: 0s - loss: 1.2818 - accuracy: 0.5169 - precision: 0.7091 - recall: 0.3014 - auc_roc: 0.8588

In [ ]:
model.save('Face_Recognition.h5')

In [ ]:
plot(history)